# Results

In [ ]:
import os
import csv
import sys
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statistics

## MINIO Data

In [ ]:
csvs = [file_name.split('_') for file_name in os.listdir('MinioData') if file_name.endswith('.csv')]
times = {}

#for item in csvs:
tmp = pd.read_csv(f'264285c4-098b-433f-90ae-e6c91eb68716_usage.csv', delimiter='|')
tmp = tmp[tmp['function'] ==  'mapper']

tmp_time = []
times['16'] = []
for elem in pd.unique(tmp['id']):
    tmp2 = tmp[tmp['id'] == elem]
    mapper_time = float(tmp2['time'].iloc[[-1]].values[0]) - float(tmp2['time'].iloc[[0]].values[0]) 
    tmp_time.append(mapper_time)
    times['16'].append(tmp2['time'].iloc[[-1]].values[0] - tmp2['time'].iloc[[0]].values[0])

print(f'Job: 16. Average Time: {sum(tmp_time)/len(tmp_time)} - Median: {statistics.median(tmp_time)}')

In [ ]:
# Box Plot
bp = go.Figure()

for k in sorted(times, key=int):
    bp.add_trace(go.Box(y=times[k], name=f'Job {k}', boxpoints='all'))

bp.show()

In [ ]:
df = pd.read_csv('264285c4-098b-433f-90ae-e6c91eb68716_usage.csv' , delimiter='|')

mappers = df[df['function'] == 'mapper']

cpu_figs = make_subplots(rows=2, cols=2, 
                         subplot_titles = ['Mapper 0','Mapper 7','Mapper 8','Mapper 12'],
                         x_title='Time (s)', y_title='CPU Usage (%)')
        
mem_figs = make_subplots(rows=2, cols=2, 
                         subplot_titles = ['Mapper 0','Mapper 1','Mapper 2','Mapper 3'],
                         x_title='Time (s)', y_title='Memory Usage (%)')
row = 1
col = 1


for id in ['0_0', '7_7', '8_8', '12_12']:
    mapper_data = mappers[mappers['id'] == id]
    
    cpu_figs.add_trace(
        go.Scatter(x=mapper_data['time'],
                   y=mapper_data['cpu_percent'],
                   showlegend=False),
        row=row, col=col
    )
    
    mem_figs.add_trace(
        go.Scatter(x=mapper_data['time'],
                   y=mapper_data['mem_percent'],
                   showlegend=False),
        row=row, col=col
    )
    
    col += 1
    if col > 2:
        col = 1
        row +=1
cpu_figs.show()
#mem_figs.show()

In [ ]:
df2 = pd.read_csv('264285c4-098b-433f-90ae-e6c91eb68716_process.csv' , delimiter='|')
df2 = df2[df2['function'] == 'mapper']
cols = ['function', 'id', 'phase', 'node'] + [col_name for col_name in df2 if col_name.startswith('ctx_') ]
df_ctx = df2[cols]
df_ctx = df_ctx[df_ctx['phase'] == 'end']
df_ctx.sort_values('ctx_involuntary')

## CERN Data

## Local Data